<a href="https://colab.research.google.com/github/Kshi-tiij/Assignment/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Check TensorFlow version
print(tf.__version__)

# Install required packages
!pip install tf-nightly
!pip install tensorflow-datasets

# Download dataset
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Load datasets
train_df = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['label', 'message'])

# Display dataset samples
print(train_df.head())
print(test_df.head())


In [ ]:
# Encode labels (ham -> 0, spam -> 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})


In [ ]:
# Tokenize the text
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['message'])

# Convert messages to sequences
train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

# Pad sequences
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

# Extract labels
train_labels = train_df['label'].values
test_labels = test_df['label'].values


In [ ]:
# Tokenize the text
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['message'])

# Convert messages to sequences
train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

# Pad sequences
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

# Extract labels
train_labels = train_df['label'].values
test_labels = test_df['label'].values


In [ ]:
# Build the model
model = keras.Sequential([
    layers.Embedding(input_dim=5000, output_dim=64, input_length=100),
    layers.LSTM(64),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Display model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_padded, train_labels,
    validation_split=0.2,
    epochs=5,
    verbose=1
)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_labels, verbose=2)
print(f'Test Accuracy: {accuracy:.2f}')


In [ ]:
# Define prediction function
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=100, padding='post', truncating='post')
    prediction = model.predict(padded)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

# Test prediction
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job! 🎉")
    else:
        print("You haven't passed yet. Keep trying. 😔")

# Run the test
test_predictions()
